In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from pathlib import Path

In [ ]:
data_folder = '../Data/QC_dev_data/counts'

In [ ]:
def read_data(data):
    data_path = Path(data)
    all_data = list(data_path.glob('*tsv'))

    dfs = []

    for data in all_data:
        df = pd.read_csv(data, sep = '\t')
        dfs.append(df)

    return dfs

In [ ]:
def process_dfs(dfs):

    clean_dfs = []
    for df in dfs:
        new_columns = [('D5_min_count', 'D13_min_count'), ('D5_median_count','D13_median_count')]
        count_cols = [['D05_R1_lib1','D05_R1_lib2','D05_R2_lib1','D05_R2_lib2', 'D05_R3_lib1', 'D05_R3_lib2'], ['D13_R1_lib1', 'D13_R1_lib2', 'D13_R2_lib1', 'D13_R2_lib2', 'D13_R3_lib1', 'D13_R3_lib2']]
        all_count_cols = count_cols[0] + count_cols[1]
        for d5, d13 in new_columns:
            if 'min' in d5:
                df[d5] = df[count_cols[0]].min(axis = 1)
                df[d13] = df[count_cols[1]].min(axis = 1)
            elif 'median' in d5:
                df[d5] = df[count_cols[0]].median(axis = 1)
                df[d13] = df[count_cols[1]].median(axis = 1)
        df['min_count'] = df[all_count_cols].min(axis = 1)
        df['median)
        df = df[['chrom', 'pos', 'ref', 'allele', 'exon', 'simplified_consequence', 'score', 'standard_error', 'D5_min_count', 'D13_min_count', 'D5_median_count', 'D13_median_count']]
        clean_dfs.append(df)
    
    return dfs

In [ ]:
def main():
    dfs = read_data(data_folder)
    final_dfs = process_dfs(dfs)

In [ ]:
main()